In [ ]:
import re
from tqdm import tqdm
import torch
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
import numpy as np

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
summary = pd.read_csv('/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/DF_SUMMARY.csv')

In [ ]:
news_press = pd.read_csv('/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/NEWS_+_PRESS_ESG_definitivo.csv')

In [ ]:
news_press = pd.read_csv('/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/NEWS_+_PRESS_ESG.csv')
summary = pd.read_csv('/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/DF_SUMMARY.csv')

In [ ]:
summary

,ticker,n_total,pct_0,pct_1,pct_2,pct_bearish,pct_bullish,pct_neutral,mean_reputation,cluster,PCA1,PCA2
0,AAPL,54655,0.177019,0.363041,0.459940,0.305279,0.267917,0.426805,0.975672,0,-0.296119,2.291074
1,ABBV,1118,0.044723,0.668157,0.287120,0.211986,0.629696,0.158318,0.954442,1,1.010288,-0.056187
2,AMZN,48131,0.290997,0.434585,0.274418,0.376763,0.302861,0.320376,0.953464,0,-0.442766,1.165137
3,AXP,2052,0.153996,0.469298,0.376706,0.236842,0.458090,0.305068,0.903144,1,0.127804,-0.853697
4,CMCSA,3060,0.117647,0.479739,0.402614,0.271895,0.457190,0.270915,0.925039,1,0.271921,-0.175843
5,COP,8165,0.851929,0.031966,0.116105,0.167544,0.289528,0.542927,0.859376,2,0.064075,-3.779208
6,CVS,6164,0.048832,0.736535,0.214633,0.371999,0.345717,0.282284,0.907844,0,-1.513328,-0.547696
7,CVX,9706,0.635071,0.078817,0.286112,0.322481,0.449310,0.228209,0.939635,1,1.739434,-0.879571
8,DIS,10405,0.125420,0.597501,0.277078,0.420375,0.113599,0.466026,0.918438,0,-2.557130,-0.075650
9,GM,16972,0.483797,0.254242,0.261961,0.351461,0.444733,0.203806,0.954700,1,1.213727,-0.111172


In [ ]:
esg_type_map = {
    0: 'Environmental',
    1: 'Social',
    2: 'Governance'
}
news_press['category'] = news_press['esg_type'].map(esg_type_map)

In [ ]:
sentiment_to_num = {
    'bullish': 0,
    'bearish': 1,
    'neutral': 2
}

news_press['sentimiento_predicho_num'] = news_press['sentimiento_predicho'].map(sentiment_to_num)

num_to_sentiment = {v: k for k, v in sentiment_to_num.items()}  # inverso
news_press['sentiment'] = news_press['sentimiento_predicho_num'].map(num_to_sentiment)

In [ ]:
news_press = news_press.rename(columns={'Publication Date': 'date'})

In [ ]:
path = '/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/NEWS_+_PRESS_ESG_definitivo.csv'
news_press.to_csv(path, index=False)

In [ ]:
news_press

,date,title,content,sector,url,ticker,text,esg_type,sentimiento_predicho,source,reputation_score,category,sentimiento_predicho_num,sentiment
0,2020-03-26,Factbox: Coronavirus cases reported at 13 of A...,Amazon.com Inc is at the front line of respon...,Technology,https://www.reuters.com/article/us-health-coro...,AMZN,Factbox: Coronavirus cases reported at 13 of A...,1,bearish,Reuters,1.0,Social,1,bearish
1,2020-03-26,"No, Jeff Bezos doesn't want your public donati...",Claims that Amazon is asking for your money ar...,Technology,https://www.cnet.com/news/no-jeff-bezos-doesnt...,AMZN,"No, Jeff Bezos doesn't want your public donati...",1,bearish,CNET,1.0,Social,1,bearish
2,2020-03-26,'They're putting us all at risk:' What it's li...,Amazon workers feel the company hasn't done en...,Technology,https://www.cnbc.com/2020/03/26/amazon-warehou...,AMZN,'They're putting us all at risk:' What it's li...,1,bearish,CNBC,1.0,Social,1,bearish
3,2020-03-26,Amazon Suspends Merchant Loan Repayments Due t...,Borrowers get some much-needed relief from the...,Technology,https://www.fool.com/investing/2020/03/26/amaz...,AMZN,Amazon Suspends Merchant Loan Repayments Due t...,1,neutral,The Motley Fool,1.0,Social,2,neutral
4,2020-03-26,Amazon adding more grocery delivery capacity d...,The e-commerce giant is also prioritizing high...,Technology,https://www.cnet.com/news/amazon-adding-more-g...,AMZN,Amazon adding more grocery delivery capacity d...,1,bullish,CNET,1.0,Social,0,bullish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427484,2025-03-23,1 Artificial Intelligence (AI) Stock Gen Zers ...,After a couple of years of making headlines in...,Technology,https://www.fool.com/investing/2025/03/23/1-ar...,AMZN,1 Artificial Intelligence (AI) Stock Gen Zers ...,2,bullish,The Motley Fool,1.0,Governance,0,bullish
427485,2025-03-23,Starbucks ‘Leaning Into Coffeehouse Culture' a...,Starbucks' effort to reestablish itself as a c...,Discreptionary Consumption,https://www.pymnts.com/news/retail/2025/starbu...,SBUX,Starbucks ‘Leaning Into Coffeehouse Culture' a...,1,bullish,PYMNTS,1.0,Social,0,bullish
427486,2025-03-23,Top 4 stocks dragging the Dow Jones Index in 2025,The blue-chip Dow Jones index has crashed this...,Technology,https://invezz.com/news/2025/03/23/top-4-stock...,AMZN,Top 4 stocks dragging the Dow Jones Index in 2...,2,bearish,Invezz,1.0,Governance,1,bearish
427487,2025-03-23,Amazon Suit Challenges Product Safety Watchdog...,Amazon has filed a lawsuit in response to a re...,Technology,https://www.pymnts.com/amazon/2025/amazon-suit...,AMZN,Amazon Suit Challenges Product Safety Watchdog...,2,bearish,PYMNTS,1.0,Governance,1,bearish


# Text Preprocessing

In this step, we clean the raw text contained in the `content` field:
- Line breaks (`\n`) and tabs (`\t`) are replaced with spaces.
- Multiple consecutive spaces are reduced to a single space.
- Leading and trailing spaces are removed.

This preprocessing is crucial to reduce noise and improve the quality of the embeddings generated in later steps.

In [ ]:
tqdm.pandas()

def preprocess_content(text):
    if not isinstance(text, str):
        return ""
    text = text.replace('\n', ' ').replace('\t', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    return text

news_press['event_text'] = news_press['content'].progress_apply(preprocess_content)

100%|██████████| 427489/427489 [00:21<00:00, 19654.68it/s]


# Text Aggregation by Ticker, Date, and ESG Category

We group all news articles by their corresponding company (`ticker`), publication date (`date`), and ESG category (`category`).
All cleaned texts are concatenated into a single entry per group.

In [ ]:
grouped = (
    news_press
    .groupby(["ticker", "date", "category"])["event_text"]
    .apply(lambda x: " ".join(x))
    .reset_index()
)

In [ ]:
grouped

,ticker,date,category,event_text
0,AAPL,2020-03-26,Governance,The coronavirus crisis won’t force IBM or thes...
1,AAPL,2020-03-26,Social,Apple Inc. (NASDAQ: AAPL) CEO Tim Cook announc...
2,AAPL,2020-03-27,Social,Apple says the app and website are meant to gi...
3,AAPL,2020-03-28,Governance,"As the U.S. economy starts to shut down, cash ..."
4,AAPL,2020-03-30,Governance,"Apple Inc's (NASDAQ: AAPL) largest supplier, F..."
...,...,...,...,...
69650,XOM,2025-03-19,Governance,ExxonMobil filed a protest notice on Wednesday...
69651,XOM,2025-03-20,Environmental,XOM questions Colonial Pipeline's plan to cut ...
69652,XOM,2025-03-21,Environmental,New Wave Offshore Energy is expected to provid...
69653,XOM,2025-03-22,Environmental,Exxon Mobil Corporation stock has remained res...


# Embedding Generation with Sentence-BERT

We transform each aggregated text into a dense vector representation (embedding) using the `all-MiniLM-L6-v2` model from Sentence Transformers (Reimers & Gurevych, 2019).

This model was selected because:
- It offers a strong trade-off between accuracy and computational efficiency.
- It produces high-quality 384-dimensional sentence embeddings.
- It is widely validated for semantic textual similarity and downstream tasks.

Embeddings are generated using GPU acceleration when available, ensuring faster processing.

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

embeddings = model.encode(
    grouped['event_text'].tolist(),
    batch_size=32,
    show_progress_bar=True,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2177 [00:00<?, ?it/s]

In [ ]:
embeddings_array = np.array(embeddings)
grouped['embedding'] = list(embeddings_array)

In [ ]:
grouped

,ticker,date,category,event_text,embedding
0,AAPL,2020-03-26,Governance,The coronavirus crisis won’t force IBM or thes...,"[-0.061546337, -0.05315133, 0.031761326, -0.03..."
1,AAPL,2020-03-26,Social,Apple Inc. (NASDAQ: AAPL) CEO Tim Cook announc...,"[0.00027809513, -0.029144509, 0.117705405, -0...."
2,AAPL,2020-03-27,Social,Apple says the app and website are meant to gi...,"[-0.015929632, 0.0119838, 0.025352007, -0.0224..."
3,AAPL,2020-03-28,Governance,"As the U.S. economy starts to shut down, cash ...","[0.032775655, -0.025250023, -0.005527121, 0.02..."
4,AAPL,2020-03-30,Governance,"Apple Inc's (NASDAQ: AAPL) largest supplier, F...","[0.021084372, -0.04204493, -0.027231807, -0.00..."
...,...,...,...,...,...
69650,XOM,2025-03-19,Governance,ExxonMobil filed a protest notice on Wednesday...,"[0.027937062, 0.05524979, 0.053883664, 0.00581..."
69651,XOM,2025-03-20,Environmental,XOM questions Colonial Pipeline's plan to cut ...,"[-0.018101593, 0.07403076, 0.039944347, -0.041..."
69652,XOM,2025-03-21,Environmental,New Wave Offshore Energy is expected to provid...,"[-0.014534758, -0.0074794795, 0.0008310969, -0..."
69653,XOM,2025-03-22,Environmental,Exxon Mobil Corporation stock has remained res...,"[-0.01221892, -0.055477124, 0.0681576, 0.06422..."


# Dimensionality Reduction with PCA

To further optimize computational efficiency and prevent overfitting, we apply **Principal Component Analysis (PCA)** to the original 384-dimensional embeddings.

We reduce the dimensionality to 100 components, preserving most of the variance while:
- Simplifying the sequential input space.
- Accelerating the training of the LSTM model.
- Improving model generalization.

PCA is a widely used technique for embedding compression without significant loss of information (Jolliffe & Cadima, 2016).

In [ ]:
n_components = 100
pca = PCA(n_components=n_components)
embeddings_reduced = pca.fit_transform(embeddings_array)

In [ ]:
grouped['embedding_reduced'] = list(embeddings_reduced)

In [ ]:
grouped

,ticker,date,category,event_text,embedding,embedding_reduced
0,AAPL,2020-03-26,Governance,The coronavirus crisis won’t force IBM or thes...,"[-0.061546337, -0.05315133, 0.031761326, -0.03...","[0.13161144, 0.21722357, -0.20482038, 0.217550..."
1,AAPL,2020-03-26,Social,Apple Inc. (NASDAQ: AAPL) CEO Tim Cook announc...,"[0.00027809513, -0.029144509, 0.117705405, -0....","[-0.1196747, 0.28669232, -0.041156188, 0.13470..."
2,AAPL,2020-03-27,Social,Apple says the app and website are meant to gi...,"[-0.015929632, 0.0119838, 0.025352007, -0.0224...","[-0.34469396, 0.13473184, -0.027780786, 0.2523..."
3,AAPL,2020-03-28,Governance,"As the U.S. economy starts to shut down, cash ...","[0.032775655, -0.025250023, -0.005527121, 0.02...","[0.108597055, -0.118303485, -0.056267083, -0.0..."
4,AAPL,2020-03-30,Governance,"Apple Inc's (NASDAQ: AAPL) largest supplier, F...","[0.021084372, -0.04204493, -0.027231807, -0.00...","[0.084087715, 0.17990188, -0.20706366, -0.0217..."
...,...,...,...,...,...,...
69650,XOM,2025-03-19,Governance,ExxonMobil filed a protest notice on Wednesday...,"[0.027937062, 0.05524979, 0.053883664, 0.00581...","[0.12804541, -0.25893778, 0.12964192, -0.17031..."
69651,XOM,2025-03-20,Environmental,XOM questions Colonial Pipeline's plan to cut ...,"[-0.018101593, 0.07403076, 0.039944347, -0.041...","[0.25796288, -0.23406321, 0.16715088, -0.04122..."
69652,XOM,2025-03-21,Environmental,New Wave Offshore Energy is expected to provid...,"[-0.014534758, -0.0074794795, 0.0008310969, -0...","[0.12106457, -0.14770606, 0.13917562, 0.138137..."
69653,XOM,2025-03-22,Environmental,Exxon Mobil Corporation stock has remained res...,"[-0.01221892, -0.055477124, 0.0681576, 0.06422...","[0.42223483, 0.006553337, -0.10097705, -0.0310..."


In [ ]:
grouped = grouped.merge(
    summary[['ticker', 'cluster']],
    on='ticker',
    how='left'
)

In [ ]:
grouped

,ticker,date,category,event_text,embedding,embedding_reduced,cluster
0,AAPL,2020-03-26,Governance,The coronavirus crisis won’t force IBM or thes...,"[-0.061546337, -0.05315133, 0.031761326, -0.03...","[0.13161144, 0.21722357, -0.20482038, 0.217550...",0
1,AAPL,2020-03-26,Social,Apple Inc. (NASDAQ: AAPL) CEO Tim Cook announc...,"[0.00027809513, -0.029144509, 0.117705405, -0....","[-0.1196747, 0.28669232, -0.041156188, 0.13470...",0
2,AAPL,2020-03-27,Social,Apple says the app and website are meant to gi...,"[-0.015929632, 0.0119838, 0.025352007, -0.0224...","[-0.34469396, 0.13473184, -0.027780786, 0.2523...",0
3,AAPL,2020-03-28,Governance,"As the U.S. economy starts to shut down, cash ...","[0.032775655, -0.025250023, -0.005527121, 0.02...","[0.108597055, -0.118303485, -0.056267083, -0.0...",0
4,AAPL,2020-03-30,Governance,"Apple Inc's (NASDAQ: AAPL) largest supplier, F...","[0.021084372, -0.04204493, -0.027231807, -0.00...","[0.084087715, 0.17990188, -0.20706366, -0.0217...",0
...,...,...,...,...,...,...,...
69650,XOM,2025-03-19,Governance,ExxonMobil filed a protest notice on Wednesday...,"[0.027937062, 0.05524979, 0.053883664, 0.00581...","[0.12804541, -0.25893778, 0.12964192, -0.17031...",1
69651,XOM,2025-03-20,Environmental,XOM questions Colonial Pipeline's plan to cut ...,"[-0.018101593, 0.07403076, 0.039944347, -0.041...","[0.25796288, -0.23406321, 0.16715088, -0.04122...",1
69652,XOM,2025-03-21,Environmental,New Wave Offshore Energy is expected to provid...,"[-0.014534758, -0.0074794795, 0.0008310969, -0...","[0.12106457, -0.14770606, 0.13917562, 0.138137...",1
69653,XOM,2025-03-22,Environmental,Exxon Mobil Corporation stock has remained res...,"[-0.01221892, -0.055477124, 0.0681576, 0.06422...","[0.42223483, 0.006553337, -0.10097705, -0.0310...",1


In [ ]:
print(grouped['embedding_reduced'].iloc[0])
print("Dimensiones del embedding_reduced:", len(grouped['embedding_reduced'].iloc[0]))

[ 1.31611437e-01  2.17223570e-01 -2.04820380e-01  2.17550755e-01
 -7.76259825e-02  3.83312255e-02  2.96809971e-02 -1.74441338e-02
  1.72618240e-01 -4.32712547e-02  7.01277405e-02  1.69277266e-02
 -7.50847757e-02  3.61490995e-03  1.64009407e-02 -1.40596196e-01
  3.40095982e-02  6.88340440e-02 -4.41622734e-03  7.33532980e-02
  9.65990648e-02  1.60627197e-02  1.41539514e-01 -3.23404968e-02
 -3.47961448e-02 -3.29508558e-02 -1.41999815e-02  2.40504257e-02
  1.79488752e-02  1.11526251e-03  1.20172482e-02  5.88777252e-02
 -5.86561486e-03 -9.94510353e-02 -9.81163383e-02 -1.13683045e-01
 -5.94952665e-02 -9.98165086e-03 -1.02615766e-01 -4.05271389e-02
 -8.45554285e-03  1.06528640e-01  1.27370685e-01 -1.26092374e-01
 -5.30235693e-02  3.16369981e-02  4.32886519e-02 -8.75473693e-02
 -1.09208658e-01  1.31195085e-03  1.43423304e-03  1.10733267e-02
 -6.27035871e-02 -9.36032385e-02 -2.37435810e-02 -7.09809884e-02
  7.68593177e-02 -2.98053641e-02  8.32595453e-02 -1.07554533e-03
 -4.11834493e-02 -1.77669

In [ ]:
def combine_embedding_and_cluster(row):
    embedding = np.array(row['embedding_reduced'])
    cluster_value = np.array([row['cluster']])
    return np.concatenate((embedding, cluster_value))

grouped['embedding_combined'] = grouped.apply(combine_embedding_and_cluster, axis=1)

print("Shape de embedding_combined:", grouped['embedding_combined'].iloc[0].shape)
print(grouped['embedding_combined'].iloc[0])

Shape de embedding_combined: (101,)
[ 1.31611437e-01  2.17223570e-01 -2.04820380e-01  2.17550755e-01
 -7.76259825e-02  3.83312255e-02  2.96809971e-02 -1.74441338e-02
  1.72618240e-01 -4.32712547e-02  7.01277405e-02  1.69277266e-02
 -7.50847757e-02  3.61490995e-03  1.64009407e-02 -1.40596196e-01
  3.40095982e-02  6.88340440e-02 -4.41622734e-03  7.33532980e-02
  9.65990648e-02  1.60627197e-02  1.41539514e-01 -3.23404968e-02
 -3.47961448e-02 -3.29508558e-02 -1.41999815e-02  2.40504257e-02
  1.79488752e-02  1.11526251e-03  1.20172482e-02  5.88777252e-02
 -5.86561486e-03 -9.94510353e-02 -9.81163383e-02 -1.13683045e-01
 -5.94952665e-02 -9.98165086e-03 -1.02615766e-01 -4.05271389e-02
 -8.45554285e-03  1.06528640e-01  1.27370685e-01 -1.26092374e-01
 -5.30235693e-02  3.16369981e-02  4.32886519e-02 -8.75473693e-02
 -1.09208658e-01  1.31195085e-03  1.43423304e-03  1.10733267e-02
 -6.27035871e-02 -9.36032385e-02 -2.37435810e-02 -7.09809884e-02
  7.68593177e-02 -2.98053641e-02  8.32595453e-02 -1.07

In [ ]:
grouped

,ticker,date,category,event_text,embedding,embedding_reduced,cluster,embedding_combined
0,AAPL,2020-03-26,Governance,The coronavirus crisis won’t force IBM or thes...,"[-0.061546337, -0.05315133, 0.031761326, -0.03...","[0.13161144, 0.21722357, -0.20482038, 0.217550...",0,"[0.1316114366054535, 0.21722356975078583, -0.2..."
1,AAPL,2020-03-26,Social,Apple Inc. (NASDAQ: AAPL) CEO Tim Cook announc...,"[0.00027809513, -0.029144509, 0.117705405, -0....","[-0.1196747, 0.28669232, -0.041156188, 0.13470...",0,"[-0.1196746975183487, 0.2866923213005066, -0.0..."
2,AAPL,2020-03-27,Social,Apple says the app and website are meant to gi...,"[-0.015929632, 0.0119838, 0.025352007, -0.0224...","[-0.34469396, 0.13473184, -0.027780786, 0.2523...",0,"[-0.34469395875930786, 0.13473184406757355, -0..."
3,AAPL,2020-03-28,Governance,"As the U.S. economy starts to shut down, cash ...","[0.032775655, -0.025250023, -0.005527121, 0.02...","[0.108597055, -0.118303485, -0.056267083, -0.0...",0,"[0.1085970550775528, -0.11830348521471024, -0...."
4,AAPL,2020-03-30,Governance,"Apple Inc's (NASDAQ: AAPL) largest supplier, F...","[0.021084372, -0.04204493, -0.027231807, -0.00...","[0.084087715, 0.17990188, -0.20706366, -0.0217...",0,"[0.08408771455287933, 0.1799018830060959, -0.2..."
...,...,...,...,...,...,...,...,...
69650,XOM,2025-03-19,Governance,ExxonMobil filed a protest notice on Wednesday...,"[0.027937062, 0.05524979, 0.053883664, 0.00581...","[0.12804541, -0.25893778, 0.12964192, -0.17031...",1,"[0.12804540991783142, -0.2589377760887146, 0.1..."
69651,XOM,2025-03-20,Environmental,XOM questions Colonial Pipeline's plan to cut ...,"[-0.018101593, 0.07403076, 0.039944347, -0.041...","[0.25796288, -0.23406321, 0.16715088, -0.04122...",1,"[0.2579628825187683, -0.23406320810317993, 0.1..."
69652,XOM,2025-03-21,Environmental,New Wave Offshore Energy is expected to provid...,"[-0.014534758, -0.0074794795, 0.0008310969, -0...","[0.12106457, -0.14770606, 0.13917562, 0.138137...",1,"[0.12106457352638245, -0.1477060616016388, 0.1..."
69653,XOM,2025-03-22,Environmental,Exxon Mobil Corporation stock has remained res...,"[-0.01221892, -0.055477124, 0.0681576, 0.06422...","[0.42223483, 0.006553337, -0.10097705, -0.0310...",1,"[0.42223483324050903, 0.006553336977958679, -0..."


In [ ]:
grouped.to_pickle("/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/EMBEDDINGS.pkl")